# Jupyter Notebook that will be used to support final Capstone Project for the IBM Data Certification

In [3]:
import numpy as np
import pandas as pd 
#Importing required libraries for Web Scraping
import urllib.request
from bs4 import BeautifulSoup
#import libraries for maps
import folium # map rendering library
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#libraries to manage JSON and web requests
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
print('Hello Capstone Project Course!!')

Hello Capstone Project Course!!


# Week-3 Assignement

## Part 1 Scraping the Web - Wikipidia Page

In [4]:
#defining the URL for the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
#Using Beatiful Soup library to get the html information
Toronto_page = urllib.request.urlopen(url)
soup = BeautifulSoup(Toronto_page, "html.parser")
print(soup.prettify())
#Saving html object to txt file
with open('url.txt', 'w', encoding='utf-8') as f_out:
    f_out.write(soup.prettify())

"
      <a dir="ltr" href="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;oldid=979555370">
       https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;oldid=979555370
      </a>
      "
     </div>
    </div>
    <div class="catlinks" data-mw="interface" id="catlinks">
     <div class="mw-normal-catlinks" id="mw-normal-catlinks">
      <a href="/wiki/Help:Category" title="Help:Category">
       Categories
      </a>
      :
      <ul>
       <li>
        <a href="/wiki/Category:Communications_in_Ontario" title="Category:Communications in Ontario">
         Communications in Ontario
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Postal_codes_in_Canada" title="Category:Postal codes in Canada">
         Postal codes in Canada
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Toronto" title="Category:Toronto">
         Toronto
        </a>
       </li>
       <li>
        <a href="/wik

In [6]:
#extracting the table from the BeatifulSoup object
city_table=soup.find("table",class_='wikitable sortable') #this is the class and tag for the zip code table
#extracting the headers from the table
table_header =city_table.find_all('th')


In [7]:
#Creating Arrays to store the data and roe headers from the table

A=[]
B=[]
C=[]
Header = []
# finding header in the table <th> tag
header_table = city_table.find_all('th')
# Append the header string values to the Header Array
for i,item in enumerate(header_table):
    
    Header.append(header_table[i].string.strip('\n'))
   
# Append the row values to the Arrays

for row in city_table.find_all('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).strip('\n'))
        B.append(cells[1].find(text=True).strip('\n'))
        C.append(cells[2].find(text=True).strip('\n'))
    

        

In [9]:
#creating DataFrame from extracted values
df = pd.DataFrame({Header[0]:A,Header[1]:B,Header[2]:C})
df=df[df["Borough"]!="Not assigned"]
df.reset_index(inplace=True,drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Part Two: Getting latidu & Longitude data

In [15]:
#using the csv file, since the geolocator option did not work for me

urlLoc = 'http://cocl.us/Geospatial_data'
dfLoc = pd.read_csv(urlLoc)
dfLoc.set_index('Postal Code',inplace = True)
dfLoc.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [40]:
#mergin both dataframes based on the postal code index
df2 = df.set_index('Postal Code')
df2 = df2.merge(dfLoc,how='left',left_index=True,right_index=True)
df2.sort_index(inplace=True)
df2.reset_index(inplace = True)
df2.head(20)



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
